In [1]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import os
import json
import yaml
import shutil
import pydicom
from tqdm import tqdm

In [2]:
df = pd.read_csv('/workspace/data/df_train_study_level_npy640_2_w_bbox.csv')
df.head()

,id,x,y,w,h,study_id,image_id,have_box,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,cv,npy_path
0,000a312787f2_image,789.28836,582.43035,1026.65662,1917.30292,5776db0cec75,000a312787f2,1,/workspace/data/train/5776db0cec75/81456c9c542...,0,1,0,0,0,/workspace/data/train_640_2/000a312787f2.png
1,000a312787f2_image,2245.91208,591.20528,1094.66162,1761.54944,5776db0cec75,000a312787f2,1,/workspace/data/train/5776db0cec75/81456c9c542...,0,1,0,0,0,/workspace/data/train_640_2/000a312787f2.png
2,000c3a3f293f_image,NaN,NaN,NaN,NaN,ff0879eb20ed,000c3a3f293f,0,/workspace/data/train/ff0879eb20ed/d8a644cc4f9...,1,0,0,0,0,/workspace/data/train_640_2/000c3a3f293f.png
3,0012ff7358bc_image,677.42216,197.97662,867.79767,999.78214,9d514ce429a7,0012ff7358bc,1,/workspace/data/train/9d514ce429a7/22897cd1daa...,0,1,0,0,1,/workspace/data/train_640_2/0012ff7358bc.png
4,0012ff7358bc_image,1792.69064,402.55250,617.02734,1204.35800,9d514ce429a7,0012ff7358bc,1,/workspace/data/train/9d514ce429a7/22897cd1daa...,0,1,0,0,1,/workspace/data/train_640_2/0012ff7358bc.png


In [3]:
df_image = df.groupby('npy_path').first().reset_index()

In [4]:
cwd = f'/workspace/exp_yolo/config_000/'
os.makedirs(cwd, exist_ok = True)

for cv in range(5):

    for train_val in ["train", "val"]:
        if train_val == "train":
            df_target = df_image[df_image.cv != cv].groupby("npy_path").first().reset_index()
        else:
            df_target = df_image[df_image.cv == cv].groupby("npy_path").first().reset_index()
        
        data_path = ''
        for i in tqdm(range(len(df_target))):
            row = df_target.iloc[i]
            new_image_path = row.npy_path.replace('train_640_2', 'yolo_data/images')
            name = new_image_path.split('/')[-1].replace('.png', '')
            label_path = f"/workspace/data/yolo_data/labels/{name}.txt"
            data_path += f'{new_image_path}\n'
        with open(os.path.join( cwd , f'{train_val}_{cv}.txt'), 'w') as f:
            f.write(data_path)
        
    data = dict(
        train =  os.path.join( cwd , f'train_{cv}.txt') ,
        val   =  os.path.join( cwd , f'val_{cv}.txt' ),
        nc    = 1,
        names = ['opacity']
        )

    with open(os.path.join( cwd , f'config_{cv}.yaml'), 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)

100%|██████████| 1264/1264 [00:00<00:00, 9293.29it/s]
